This code is developped in the context on my internship with Pablo JENSEN regarding economic polarisation in France during the post-1980 globalisation.

The purpose of this notebook is to expose different metrics for measuring economic polarisation (mostly from INSEE data)

In [1]:
### Importing libraries ###
import numpy as np
import json
import pandas as pd
import geopandas as gpd
import datetime


import folium
from folium.plugins import TimeSliderChoropleth
import branca.colormap as cm


In [2]:
### Defining path ###

path = "/home/avel/Documents/Travail/2019--20 : Stage Jensen/france-polarisation-code/ZE level cartography/"

In [3]:
### Importing & formatting geographic data

    ## Importing communal data
communal_geo = gpd.read_file(path + "Data/communes-version-simplifiee.geojson")


    ## Building ZE1990 geography
with open(path + "Data/ZE1990.csv") as ZE1990_comp_csv :
    ZE1990_comp = pd.read_csv(ZE1990_comp_csv, header=1, usecols=[0,4], names=["Code","ZE1990"])

ZE1990_geo = communal_geo.merge(ZE1990_comp,right_on="Code",left_on="code")
ZE1990_geo = ZE1990_geo[["ZE1990","geometry"]]
ZE1990_geo = ZE1990_geo[ZE1990_geo.geometry.notnull()]
ZE1990_geo = ZE1990_geo.dissolve("ZE1990")
##ZE1990_geo = ZE1990_geo.reset_index()


    ## Building ZE2010 geography
with open(path + "Data/ZE2010.csv") as ZE2010_comp_csv :
    ZE2010_comp = pd.read_csv(ZE2010_comp_csv, header=5, usecols=[0,2], names=["Code","ZE2010"])

ZE2010_geo = communal_geo.merge(ZE2010_comp,right_on="Code",left_on="code")
ZE2010_geo = ZE2010_geo[["ZE2010","geometry"]]
ZE2010_geo = ZE2010_geo[ZE2010_geo.geometry.notnull()]
ZE2010_geo = ZE2010_geo.dissolve("ZE2010")
##ZE2010_geo = ZE2010_geo.reset_index()

In [4]:
### Drawing ZE geography map

m = folium.Map(
    location=[47,2],
    zoom_start=6,
    tiles="cartodbpositron"
)
folium.TileLayer('openstreetmap').add_to(m)

folium.GeoJson(ZE1990_geo,name = "ZE 1990").add_to(m)
folium.GeoJson(ZE2010_geo, name = "ZE 2010").add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save("ZE geography.html")

In [5]:
### Importing times series from 2001 to 2008
 
dates = range(2001,2009)

dUC_series_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)
med_series_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)
intQ_series_ZE1990 = pd.DataFrame(columns = ZE1990_geo.index, index=dates)

print("loading data...")
for i in dates:
    with open(path + "Data/FILO" + str(i) + "_DEC_ZE1990.csv") as ZE_revenue_csv :

        ## load from 2001 to 2006        
        if i in range(2001,2007):
            ZE_data = pd.read_csv(ZE_revenue_csv, header = 6, usecols=[0,5,7,18], names = ["ZE1990","nUC","med","intQ"])

        ## load from 2007 to 2008
        elif i in range(2007,2009):
            ZE_data = pd.read_csv(ZE_revenue_csv, header = 6, usecols=[0,3,5,16], names = ["ZE1990","nUC","med","intQ"])
            
        ZE_data = ZE_data.set_index("ZE1990")

        dUC_series_ZE1990.loc[i]=ZE_data["nUC"].div(ZE1990_geo.geometry.area)/(110.574**2) ##Conversion from latlon to km**2 surface
        med_series_ZE1990.loc[i]=ZE_data["med"]
        intQ_series_ZE1990.loc[i]=ZE_data["intQ"]

print("OK")

loading data...
OK


In [6]:
### Importing times series from 2009 to 2016
 
dates = range(2009,2017)

dUC_series_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)
med_series_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)
intQ_series_ZE2010 = pd.DataFrame(columns = ZE2010_geo.index, index=dates)


print("loading data...")
for i in dates:
    with open(path + "Data/FILO" + str(i) + "_DEC_ZE2010.csv") as ZE_revenue_csv :
        
        ## load from 2009 to 2011        
        if i in range(2009,2012):
            ZE_data = pd.read_csv(ZE_revenue_csv, header = 6, usecols=[0,3,5,16], names = ["ZE2010","nUC","med","intQ"])
        
        ## load from 2012 to 2016
        elif i in range(2012,2017):
            ZE_data = pd.read_csv(ZE_revenue_csv, header = 5, usecols=[0,4,7,18], names = ["ZE2010","nUC","med","intQ"])

        ZE_data = ZE_data.set_index("ZE2010")

        dUC_series_ZE2010.loc[i]=ZE_data["nUC"].div(ZE2010_geo.geometry.area)/(110.574**2) ##Conversion from latlon to km**2 surface
        med_series_ZE2010.loc[i]=ZE_data["med"]
        intQ_series_ZE2010.loc[i]=ZE_data["intQ"]
        
print("OK")

loading data...
OK


In [7]:
def style(df, datatype) :
    
    valid_datatype = {"dUC","med","intQ"}
    if datatype not in valid_datatype:
        raise ValueError("datatype must be one of %r." % valid_datatype)
    
    styledict = {}
    df = df.to_dict()
    min_color=1000000
    max_color=0
    
    for ZE, timeseries in df.items():
        min_color = min(min_color, min(timeseries.values()))
        max_color = max(max_color, max(timeseries.values()))
    
    if datatype == "dUC":
        cmap = cm.linear.YlGn_09.scale(min_color, max_color)       
    elif datatype == "med":
        cmap = cm.linear.YlOrRd_09.scale(min_color, max_color)
    elif datatype == "intQ":
        cmap = cm.linear.PuRd_09.scale(min_color, max_color)

    for ZE, timeseries in df.items():
        styledict[ZE] = {}
        for year in timeseries.keys() :
            d = {}
            d["color"] = cmap(df[ZE][year])
            d["opacity"] = 0.8
            
            time = datetime.datetime(year,1,1)
            time = int(time.strftime('%s'))
            
            styledict[ZE][time] = d
    return(styledict,cmap)

In [8]:
### Drawing UC log-density Timeslider choropleth for ZE1990 data

m = folium.Map(
    location=[47,2],
    zoom_start=6,
    tiles="cartodbpositron"
    )
folium.TileLayer('openstreetmap').add_to(m)

[styledict,cmap] = style(dUC_series_ZE1990.applymap(lambda x : np.log(x)),"dUC")
 
TimeSliderChoropleth(
    ZE1990_geo,
    styledict=styledict,
    name="Log-densité des Unités de Consommation - choropleth"

).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


m.save("Log-densité des Unités de Consommation - ZE - TimeSlider 2001 to 2008.html")

In [9]:
### Drawing UC log-density Timeslider choropleth for ZE2010 data

m = folium.Map(
    location=[47,2],
    zoom_start=6,
    tiles="cartodbpositron"
    )
folium.TileLayer('openstreetmap').add_to(m)

[styledict,cmap] = style(dUC_series_ZE2010.applymap(lambda x : np.log(x)),"dUC")
 
TimeSliderChoropleth(
    ZE2010_geo,
    styledict=styledict,
    name="Log-densité des Unités de Consommation - choropleth"

).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


m.save("Log-densité des Unités de Consommation - ZE - TimeSlider 2009 to 2016.html")

In [23]:
### Drawing UC log-density choropleth for 2016


m = folium.Map(
    location=[47,2],
    zoom_start=6,
    tiles="cartodbpositron"
    )
folium.TileLayer('openstreetmap').add_to(m)

df = pd.DataFrame()
df["dUC_2016"]=med_series_ZE2010.loc[2016]
df = df.reset_index()

folium.Choropleth(
    geo_data = ZE2010_geo,
    data = df,
    key_on = "feature.id",
    columns = ["ZE2010","dUC_2016"],
    fill_color ='YlGn',
    legend_name = 'Log(densité des UC (km**-2))',
    name="Log-densité des Unités de Consommation - choropleth"
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


m.save("Log-densité des Unités de Consommation - ZE - 2016.html")

In [10]:
### Drawing median revenue Timeslider choropleth for ZE1990 data

m = folium.Map(
    location=[47,2],
    zoom_start=6,
    tiles="cartodbpositron"
    )
folium.TileLayer('openstreetmap').add_to(m)

[styledict,cmap] = style(med_series_ZE1990,"med")
 
TimeSliderChoropleth(
    ZE1990_geo,
    styledict=styledict,
    name="Revenu médian - choropleth"

).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


m.save("Revenu médian - ZE - TimeSlider 2001 to 2008.html")

In [11]:
### Drawing median revenue Timeslider choropleth for ZE2010 data

m = folium.Map(
    location=[47,2],
    zoom_start=6,
    tiles="cartodbpositron"
    )
folium.TileLayer('openstreetmap').add_to(m)

[styledict,cmap] = style(med_series_ZE2010,"med")

TimeSliderChoropleth(
    ZE2010_geo,
    styledict=styledict,
    name="Revenu médian - choropleth"


).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


m.save("Revenu médian - ZE - TimeSlider 2009 to 2016.html")

In [12]:
### Drawing median revenue choropleth for 2016


m = folium.Map(
    location=[47,2],
    zoom_start=6,
    tiles="cartodbpositron"
    )
folium.TileLayer('openstreetmap').add_to(m)

df = pd.DataFrame()
df["med_2016"]=med_series_ZE2010.loc[2016]
df = df.reset_index()

folium.Choropleth(
    geo_data = ZE2010_geo,
    data = df,
    key_on = "feature.id",
    columns = ["ZE2010","med_2016"],
    fill_color ='YlOrRd',
    legend_name = 'Revenu médian (Euros)',
    name = "Revenu médian - choropleth"
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


m.save("Revenu médian - ZE - 2016.html")

In [13]:
print(intQ_series_ZE1990.loc[:,intQ_series_ZE1990.isnull().any()])

## Il manque le rapport interdécile pour la ZE1990 4111 !

intQ_series_ZE1990 = intQ_series_ZE1990.dropna(axis=1)

ZE1990 4111
2001    NaN
2002    NaN
2003    NaN
2004    NaN
2005    NaN
2006    NaN
2007      7
2008    5.6


In [14]:
### Drawing intedecile quotient Timeslider choropleth for ZE1990 data


m = folium.Map(
    location=[47,2],
    zoom_start=6,
    tiles="cartodbpositron"
    )
folium.TileLayer('openstreetmap').add_to(m)

[styledict,cmap] = style(med_series_ZE1990,"intQ")

TimeSliderChoropleth(
    ZE1990_geo,
    styledict=styledict,
    name="Quotient interdécile - choropleth"
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


m.save("Quotient interdécile - ZE - TimeSlider 2001 to 2008.html")

In [15]:
### Drawing intedecile quotient Timeslider choropleth for ZE2010 data


m = folium.Map(
    location=[47,2],
    zoom_start=6,
    tiles="cartodbpositron"
    )
folium.TileLayer('openstreetmap').add_to(m)

[styledict,cmap] = style(med_series_ZE2010,"intQ")
 
TimeSliderChoropleth(
    ZE2010_geo,
    styledict=styledict,
    name="Quotient interdécile - choropleth"
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


m.save("Quotient interdécile - ZE - TimeSlider 2009 to 2016.html")

In [16]:
### Drawing interdecile quotient choropleth for 2016


m = folium.Map(
    location=[47,2],
    zoom_start=6,
    tiles="cartodbpositron"
    )
folium.TileLayer('openstreetmap').add_to(m)

df = pd.DataFrame()
df["intQ_2016"]=intQ_series_ZE2010.loc[2016]
df = df.reset_index()

folium.Choropleth(
    geo_data = ZE2010_geo,
    data = df,
    key_on = "feature.id",
    columns = ["ZE2010","intQ_2016"],
    fill_color='PuRd',
    legend_name='Quotient interdécile',
    name="Quotient interdécile - choropleth"
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


m.save("Quotient interdécile - ZE - 2016.html")